<a href="https://colab.research.google.com/github/dchlseo/DataScienceProjects/blob/main/TensorFlow/01_tf_linear_reg_diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import load_diabetes
import pandas as pd
import numpy as np
import tensorflow as tf

diabetes = load_diabetes()
df = pd.DataFrame(diabetes.data,
                  columns=diabetes.feature_names,
                  dtype=np.float32)
df['const'] = np.ones(df.shape[0])
df.tail(3)

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,const
439,0.041708,0.050680,-0.015906,0.017293,-0.037344,-0.013840,-0.024993,-0.011080,-0.046883,0.015491,1.0
440,-0.045472,-0.044642,0.039062,0.001215,0.016318,0.015283,-0.028674,0.026560,0.044529,-0.025930,1.0
441,-0.045472,-0.044642,-0.073030,-0.081413,0.083740,0.027809,0.173816,-0.039493,-0.004222,0.003064,1.0


$
w^{\ast} = (X^TX)^{-1} X^T y
$

- where $X$ is the matrix of features and
$y$ is the vector of target values.
- This equation provides a way to calculate the weights ($w^{\ast}$) that minimize the cost function for linear regression without the need for iterative optimization like gradient descent.

In [ ]:
# Features
X = df.values

# target vector & reshaped
y = np.expand_dims(diabetes.target, axis=1)  # (442, 1)

# X transpose
XT = tf.transpose(X)

# get weights
w = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT, X)), XT), y)

# same operation using numpy instead of tf
# w = np.linalg.inv(XT @ X) @ XT @ y

# prediction
y_pred = tf.matmul(X, w)

In [ ]:
print('Predicted y: ', y_pred[0].numpy(), 'Real y: ', y[0])
print('Predicted y: ', y_pred[19].numpy(), 'Real y: ', y[19])
print('Predicted y: ', y_pred[64].numpy(), 'Real y: ', y[64])

Predicted y:  [206.11667747] Real y:  [151.]
Predicted y:  [124.01754101] Real y:  [168.]
Predicted y:  [113.79577042] Real y:  [71.]


Errors seem quite large. Let's try SGD.

In [ ]:
lr = 0.03
num_iter = 100 # epochs
w_init = tf.random.normal((X.shape[-1], 1), dtype=tf.float64) # weight is initialized as random num from gaussian normal distribution.
w = tf.Variable(w_init)

In [ ]:
for i in range(num_iter):
  with tf.GradientTape() as tape:
    y_hat = tf.matmul(X, w)
    loss = tf.reduce_mean(tf.square(y-y_hat)) # MSE

  dw = tape.gradient(loss, w)
  w.assign_sub(lr * dw)

In [ ]:
print('Predicted y: ', y_hat[0].numpy(), 'Real y: ', y[0])
print('Predicted y: ', y_hat[19].numpy(), 'Real y: ', y[19])
print('Predicted y: ', y_hat[64].numpy(), 'Real y: ', y[64])

Predicted y:  [153.10214663] Real y:  [151.]
Predicted y:  [149.72886627] Real y:  [168.]
Predicted y:  [150.65393372] Real y:  [71.]
